In [72]:
import re
import string
from collections import Counter
import numpy as np
import nltk
import pandas as pd
from pattern.en import lemma, lexeme  # Ensure this import works correctly or use an alternative


In [73]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [87]:
# words
w = []

# reading text file
with open('final.txt', 'r', encoding="utf8") as f:
    file_name_data = f.read()
    file_name_data = file_name_data.lower()
    w = re.findall(r'\w+', file_name_data)

# vocabulary
main_set = set(w)


In [88]:
# Functions to count the frequency of the words in the whole text file
def counting_words(words):
    word_count = {}
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count

In [89]:
# Calculating the probability of each word
def prob_cal(word_count_dict):
    probs = {}
    m = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key] / m
    return probs

# LemmWord: extracting and adding root word i.e., Lemma using pattern module
def LemmWord(word):
    return list(lexeme(wd) for wd in word.split())[0]


In [90]:
# LemmWord: extracting and adding
# root word i.e.Lemma using pattern module
import pattern
from pattern.en import lemma, lexeme
from nltk.stem import WordNetLemmatizer


def LemmWord(word):
	return list(lexeme(wd) for wd in word.split())[0]


In [91]:
# Deleting letters from the words
def DeleteLetter(word):
    delete_list = []
    split_list = []
    for i in range(len(word)):
        split_list.append((word[0:i], word[i:]))
    for a, b in split_list:
        delete_list.append(a + b[1:])
    return delete_list


In [92]:
# Switching two letters in a word
def Switch_(word):
    split_list = []
    switch_l = []
    for i in range(len(word)):
        split_list.append((word[0:i], word[i:]))
    switch_l = [a + b[1] + b[0] + b[2:] for a, b in split_list if len(b) >= 2]
    return switch_l


In [93]:
def Replace_(word):
    split_l = []
    replace_list = []
    for i in range(len(word)):
        split_l.append((word[0:i], word[i:]))
    alphs = 'abcdefghijklmnopqrstuvwxyz'
    replace_list = [a + l + (b[1:] if len(b) > 1 else '') for a, b in split_l if b for l in alphs]
    return replace_list


In [94]:
def insert_(word):
    split_l = []
    insert_list = []
    for i in range(len(word) + 1):
        split_l.append((word[0:i], word[i:]))
    alphs = 'abcdefghijklmnopqrstuvwxyz'
    insert_list = [a + l + b for a, b in split_l for l in alphs]
    return insert_list

In [95]:
# Collecting all the words in a set (so that no word will repeat)
def colab_1(word, allow_switches=True):
    colab_1 = set()
    colab_1.update(DeleteLetter(word))
    if allow_switches:
        colab_1.update(Switch_(word))
    colab_1.update(Replace_(word))
    colab_1.update(insert_(word))
    return colab_1

In [96]:
# Collecting words using by allowing switches
def colab_2(word, allow_switches=True):
    colab_2 = set()
    edit_one = colab_1(word, allow_switches=allow_switches)
    for w in edit_one:
        if w:
            edit_two = colab_1(w, allow_switches=allow_switches)
            colab_2.update(edit_two)
    return colab_2

# Only storing those values which are in the vocab
def get_corrections(word, probs, vocab, n=2):
    suggested_word = []
    best_suggestion = []
    suggested_word = list((word in vocab and word) or colab_1(word).intersection(vocab) or colab_2(word).intersection(vocab))
    best_suggestion = [[s, probs[s]] for s in list(reversed(suggested_word))]
    return best_suggestion

In [97]:
# Input
my_word = input("Enter any word: ")

# Counting word function
word_count = counting_words(w)  # Calculate word count from the list of words, not the set

# Calculating probability
probs = prob_cal(word_count)  # Use correct function name

# Only storing correct words
tmp_corrections = get_corrections(my_word, probs, main_set, 2)
for i, word_prob in enumerate(tmp_corrections):
    if i < 3:
        print(word_prob[0])
    else:
        break



Enter any word: novep 
novels
novel
noveau


## different model

In [25]:
def read_corpus(filename):
  with open(filename, "r") as file:
    lines = file.readlines()
    words = []
    for line in lines:
      words += re.findall(r'\w+', line.lower())

  return words

In [26]:
words = read_corpus("./final.txt")
print(f"There are {len(words)} total words in the corpus")

There are 1164823 total words in the corpus


In [27]:
vocabs = set(words)
print(f"There are {len(vocabs)} unique words in the vocabulary")

There are 66684 unique words in the vocabulary


In [28]:
word_counts = Counter(words)
print(word_counts["love"])

487


In [29]:
total_word_count = float(sum(word_counts.values()))
word_probas = {word: word_counts[word] / total_word_count for word in word_counts.keys()}

In [30]:
print(word_probas["love"])

0.00041808927193230216


In [31]:
def split(word):
  return [(word[:i], word[i:]) for i in range(len(word) + 1)]

In [32]:
print(split("trash"))

[('', 'trash'), ('t', 'rash'), ('tr', 'ash'), ('tra', 'sh'), ('tras', 'h'), ('trash', '')]


In [33]:
def delete(word):
  return [l + r[1:] for l,r in split(word) if r]

In [34]:
print(delete("trash"))

['rash', 'tash', 'trsh', 'trah', 'tras']


In [35]:
def swap(word):
  return [l + r[1] + r[0] + r[2:] for l, r in split(word) if len(r)>1]

In [36]:
print(swap("trash"))

['rtash', 'tarsh', 'trsah', 'trahs']


In [37]:
string.ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'

In [38]:
def replace(word):
  letters = string.ascii_lowercase
  return [l + c + r[1:] for l, r in split(word) if r for c in letters]

In [39]:
print(replace("trash"))

['arash', 'brash', 'crash', 'drash', 'erash', 'frash', 'grash', 'hrash', 'irash', 'jrash', 'krash', 'lrash', 'mrash', 'nrash', 'orash', 'prash', 'qrash', 'rrash', 'srash', 'trash', 'urash', 'vrash', 'wrash', 'xrash', 'yrash', 'zrash', 'taash', 'tbash', 'tcash', 'tdash', 'teash', 'tfash', 'tgash', 'thash', 'tiash', 'tjash', 'tkash', 'tlash', 'tmash', 'tnash', 'toash', 'tpash', 'tqash', 'trash', 'tsash', 'ttash', 'tuash', 'tvash', 'twash', 'txash', 'tyash', 'tzash', 'trash', 'trbsh', 'trcsh', 'trdsh', 'tresh', 'trfsh', 'trgsh', 'trhsh', 'trish', 'trjsh', 'trksh', 'trlsh', 'trmsh', 'trnsh', 'trosh', 'trpsh', 'trqsh', 'trrsh', 'trssh', 'trtsh', 'trush', 'trvsh', 'trwsh', 'trxsh', 'trysh', 'trzsh', 'traah', 'trabh', 'trach', 'tradh', 'traeh', 'trafh', 'tragh', 'trahh', 'traih', 'trajh', 'trakh', 'tralh', 'tramh', 'tranh', 'traoh', 'traph', 'traqh', 'trarh', 'trash', 'trath', 'trauh', 'travh', 'trawh', 'traxh', 'trayh', 'trazh', 'trasa', 'trasb', 'trasc', 'trasd', 'trase', 'trasf', 'trasg', 

In [40]:
def insert(word):
  letters = string.ascii_lowercase
  return [l + c + r for l, r in split(word) for c in letters]

In [41]:
print(insert("trash"))

['atrash', 'btrash', 'ctrash', 'dtrash', 'etrash', 'ftrash', 'gtrash', 'htrash', 'itrash', 'jtrash', 'ktrash', 'ltrash', 'mtrash', 'ntrash', 'otrash', 'ptrash', 'qtrash', 'rtrash', 'strash', 'ttrash', 'utrash', 'vtrash', 'wtrash', 'xtrash', 'ytrash', 'ztrash', 'tarash', 'tbrash', 'tcrash', 'tdrash', 'terash', 'tfrash', 'tgrash', 'thrash', 'tirash', 'tjrash', 'tkrash', 'tlrash', 'tmrash', 'tnrash', 'torash', 'tprash', 'tqrash', 'trrash', 'tsrash', 'ttrash', 'turash', 'tvrash', 'twrash', 'txrash', 'tyrash', 'tzrash', 'traash', 'trbash', 'trcash', 'trdash', 'treash', 'trfash', 'trgash', 'trhash', 'triash', 'trjash', 'trkash', 'trlash', 'trmash', 'trnash', 'troash', 'trpash', 'trqash', 'trrash', 'trsash', 'trtash', 'truash', 'trvash', 'trwash', 'trxash', 'tryash', 'trzash', 'traash', 'trabsh', 'tracsh', 'tradsh', 'traesh', 'trafsh', 'tragsh', 'trahsh', 'traish', 'trajsh', 'traksh', 'tralsh', 'tramsh', 'transh', 'traosh', 'trapsh', 'traqsh', 'trarsh', 'trassh', 'tratsh', 'traush', 'travsh',

In [42]:
def edit1(word):
  return set(delete(word) + swap(word) + replace(word) + insert(word))

In [43]:
print(edit1("trash"))

{'tfash', 'tmash', 'trashq', 'trasbh', 'trashd', 'terash', 'trazh', 'traksh', 'trzash', 'tragsh', 'traseh', 'tralsh', 'tmrash', 'trashe', 'trashn', 'trasdh', 'prash', 'ttash', 'tkrash', 'trasmh', 'krash', 'traeh', 'wrash', 'trvash', 'ztrash', 'tprash', 'trwash', 'frash', 'trish', 'otrash', 'trafsh', 'traush', 'traqsh', 'trnsh', 'trach', 'truash', 'tnash', 'trashw', 'trasu', 'rrash', 'triash', 'trsash', 'trmash', 'trasi', 'tradh', 'thash', 'txash', 'traph', 'trqash', 'trrash', 'trasq', 'tratsh', 'traslh', 'htrash', 'trpsh', 'trafh', 'qrash', 'tralh', 'toash', 'trasa', 'tiash', 'trass', 'trawh', 'tvash', 'trashx', 'ntrash', 'travsh', 'xrash', 'trasuh', 'trahh', 'turash', 'trqsh', 'ftrash', 'trashu', 'trashs', 'trashi', 'trashz', 'drash', 'tcash', 'trasih', 'trashy', 'tdrash', 'erash', 'trasoh', 'trasxh', 'trasy', 'dtrash', 'trask', 'trashr', 'tzash', 'brash', 'trasgh', 'trasw', 'tqrash', 'atrash', 'trassh', 'vrash', 'qtrash', 'traxsh', 'lrash', 'ptrash', 'trlsh', 'trhash', 'tradsh', 'ura

In [44]:
def edit2(word):
  return set(e2 for e1 in edit1(word) for e2 in edit1(e1))

In [45]:
print(edit2("trash"))

{'uzash', 'trltash', 'tturash', 'mraxh', 'kraszh', 'trtxh', 'wtrasp', 'tyrapsh', 'trisn', 'tralsp', 'traoch', 'trapshl', 'tirarh', 'tzansh', 'dsash', 'traesch', 'trawsm', 'trrsph', 'ertsh', 'tnrashr', 'trqshv', 'strabh', 'jtras', 'ntrah', 'trashji', 'tpksh', 'tvasf', 'xwrash', 'rraxh', 'traw', 'traswg', 'txrashr', 'atrakh', 'trxasl', 'altrash', 'truasjh', 'trfpash', 'ftrasa', 'tplsh', 'trlshr', 'irasu', 'tryabh', 'tralshe', 'taraqh', 'ernsh', 'trahshx', 'traschv', 'gtrrash', 'tyasn', 'yrasn', 'grarsh', 'htzash', 'tradx', 'mtrasjh', 'trgasi', 'tgasdh', 'txravsh', 'tryoh', 'traxshk', 'tpashd', 'txashx', 'txrush', 'trkjh', 'tradsjh', 'triwsh', 'mtrcsh', 'trcasg', 'tvraph', 'trasgp', 'traossh', 'utrashe', 'tzrass', 'tdrah', 'ftraseh', 'traksc', 'vtraxsh', 'trashke', 'trcasuh', 'xcrash', 'trvabh', 'drabh', 'etzash', 'trashqh', 'tprasgh', 'tbashd', 'truzsh', 'nrashw', 'trkqash', 'thasp', 'urhash', 'arasoh', 'trnasmh', 'btrarh', 'ztrvash', 'ubrash', 'gtrasrh', 'tprmash', 'trvsbh', 'ertrash', 

In [46]:
def correct_spelling(word, vocabulary, word_probabilities):
  if word in vocabulary:
    print(f"{word} is already correctly spelt")
    return

  suggestions = edit1(word) or edit2(word) or [word]
  best_guesses = [w for w in suggestions if w in vocabulary]
  return [(w, word_probabilities[w]) for w in best_guesses]

In [47]:
word = "famile"
corrections = correct_spelling(word, vocabs, word_probas)

if corrections:
  print(corrections)
  probs = np.array([c[1] for c in corrections])
  best_ix = np.argmax(probs)
  correct = corrections[best_ix][0]
  print(f"{correct} is suggested for {word}")

[('family', 0.00018286040024965167), ('familer', 1.716999063377011e-06), ('familes', 1.716999063377011e-06), ('familea', 8.584995316885054e-07), ('famine', 2.5754985950655166e-06)]
family is suggested for famile


In [48]:
class SpellChecker(object):

  def __init__(self, corpus_file_path):
    with open(corpus_file_path, "r") as file:
      lines = file.readlines()
      words = []
      for line in lines:
        words += re.findall(r'\w+', line.lower())

    self.vocabs = set(words)
    self.word_counts = Counter(words)
    total_words = float(sum(self.word_counts.values()))
    self.word_probas = {word: self.word_counts[word] / total_words for word in self.vocabs}

  def _level_one_edits(self, word):
    letters = string.ascii_lowercase
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [l + r[1:] for l,r in splits if r]
    swaps = [l + r[1] + r[0] + r[2:] for l, r in splits if len(r)>1]
    replaces = [l + c + r[1:] for l, r in splits if r for c in letters]
    inserts = [l + c + r for l, r in splits for c in letters]

    return set(deletes + swaps + replaces + inserts)

  def _level_two_edits(self, word):
    return set(e2 for e1 in self._level_one_edits(word) for e2 in self._level_one_edits(e1))

  def check(self, word):
    candidates = self._level_one_edits(word) or self._level_two_edits(word) or [word]
    valid_candidates = [w for w in candidates if w in self.vocabs]
    return sorted([(c, self.word_probas[c]) for c in valid_candidates], key=lambda tup: tup[1], reverse=True)


In [49]:
checker = SpellChecker("./final.txt")

In [50]:
checker.check("sentense")

[('sentence', 2.3179487355589647e-05)]